### Imports

In [ ]:
import os
import glob
import argparse
import time
from PIL import Image
import numpy as np
import PIL
import h5py
import cv2

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from tensorflow.keras.models import load_model
from layers import BilinearUpSampling2D
from loss import depth_loss_function
from utils import predict, display_images
from matplotlib import pyplot as plt

### Load Model

In [ ]:
# Argument Parser
parser = argparse.ArgumentParser(description='High Quality Monocular Depth Estimation via Transfer Learning')
parser.add_argument('--model', default='nyu.h5', type=str, help='Trained Keras model file.')
parser.add_argument('--input', default='my_examples/*.jpg', type=str, help='Input filename or folder.')
args, unknown = parser.parse_known_args()

# Custom object needed for inference and training
start = time.time()
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': depth_loss_function}

print('Loading model...')

# Load model into GPU / CPU
model = load_model(args.model, custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format(args.model))

### Predict Heatmap Image

In [ ]:
def load_images_with_resize(image_files):
    loaded_images = []
    for file in image_files:
        im = Image.open( file )
        im = im.resize((640, 480), PIL.Image.ANTIALIAS)
        x = np.clip(np.asarray(im, dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)

def DepthNorm(x, maxDepth):
    return maxDepth / x

def predict(model, images, minDepth=10, maxDepth=1000, batch_size=2):
    # Support multiple RGBs, one RGB image, even grayscale 
    if len(images.shape) < 3: images = np.stack((images,images,images), axis=2)
    if len(images.shape) < 4: images = images.reshape((1, images.shape[0], images.shape[1], images.shape[2]))
    # Compute predictions
    print(images[0].shape)
    predictions = model.predict(images, batch_size=batch_size)
    print(predictions[0].shape)
    # Put in expected range
    return np.clip(DepthNorm(predictions, maxDepth=1000), minDepth, maxDepth) / maxDepth

# Input images
inputs = load_images_with_resize( glob.glob(args.input) )
print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))

# Compute results
outputs = predict(model, inputs)

end = time.time()
print('It took: ', end - start)

### Display Image

In [ ]:
# Normal Image
rgb_image = cv2.imread('my_examples/img-office.jpg')
plt.figure(figsize=(10,5))
out = plt.imshow(rgb_image)

In [ ]:
# Display Ground-Truth Image
f = h5py.File('nyu_depth_v2_labeled.mat', 'r')
nyu_depths = f['depths']
nyu_depth_image = nyu_depths[0]
nyu_depth_image = cv2.rotate(nyu_depth_image, cv2.ROTATE_90_CLOCKWISE)
nyu_depth_image = cv2.flip(nyu_depth_image, 1)

# Delete every other row and column of the array

plt.imshow(nyu_depth_image, cmap='plasma', interpolation='nearest')

In [ ]:
# Display Predicted Image
def normalize_image(image):
    min = np.min(image)
    max = np.max(image)
    return (image - min) / (max - min)

def display(image):
    print("Value Range:", image.min(), "-", image.max())
    print(image.shape)
    plt.figure(figsize=(10,5))
    out = plt.imshow(image, cmap='plasma')
    out = plt.colorbar()

output = normalize_image(np.asarray(outputs[0])) # This output can be messed with however now
display(output)
# plt.savefig('results.png')

In [ ]:
# Display the two images as a function between each other
x_axis = output.flatten()
y_axis = nyu_depth_image.flatten()
plt.plot(x_axis, y_axis)
plt.title('title name')
plt.xlabel('x_axis name')
plt.ylabel('y_axis name')
plt.show()